In [10]:
import numpy as np
import os
import pandas as pd
from tkinter import *
from PIL import Image, ImageTk

In [ ]:
# https://www.boardgameanalysis.com/the-102-ways-to-win-at-catan-part-i/

In [63]:
# Ways to win
waysToWin = 0
winScenario = []

for longestRoad in range(1 + 1):
    for largestArmy in range(1 + 1):
        for victoryPoint in range(5 + 1):
            for settlement in range(4 + 1):
                for city in range(settlement + 1):
                    for newSettlement in range(city + 1):

                        totalScore = settlement + city + newSettlement + victoryPoint + longestRoad * 2 + largestArmy * 2
                        if (totalScore >= 10 and totalScore <= 11):
                            waysToWin += 1 
#                             print('ways to win: #', waysToWin, 'score:', totalScore)
#                             print(
#                                 'victory point:', victoryPoint,
#                                 'longest road:', longestRoad*2,
#                                 'largest army:', largestArmy*2,
#                                 'settlement:', settlement - city + newSettlement, 
#                                 'city:', city
#                             )
                            winScenarioText = str(
                                    victoryPoint*10000 
                                    + longestRoad*1*1000 
                                    + largestArmy*1*100 
                                    + (settlement - city + newSettlement)*10 
                                    + city
                                ).zfill(5)
                            winScenario.append(winScenarioText)
# Remove duplicated scenarios                     
winScenario = list(dict.fromkeys(winScenario))
winScenario

['00043',
 '00024',
 '00034',
 '10033',
 '10043',
 '10014',
 '10024',
 '20023',
 '20033',
 '20042',
 '20004',
 '20014',
 '30032',
 '30013',
 '30023',
 '30042',
 '30004',
 '40022',
 '40032',
 '40003',
 '40013',
 '40041',
 '50012',
 '50022',
 '50031',
 '50003',
 '50041',
 '00123',
 '00133',
 '00142',
 '00104',
 '00114',
 '10132',
 '10113',
 '10123',
 '10142',
 '10104',
 '20122',
 '20132',
 '20103',
 '20113',
 '20141',
 '30112',
 '30122',
 '30131',
 '30103',
 '30141',
 '40121',
 '40102',
 '40112',
 '40131',
 '40140',
 '50111',
 '50121',
 '50102',
 '50130',
 '50140',
 '01023',
 '01033',
 '01042',
 '01004',
 '01014',
 '11032',
 '11013',
 '11023',
 '11042',
 '11004',
 '21022',
 '21032',
 '21003',
 '21013',
 '21041',
 '31012',
 '31022',
 '31031',
 '31003',
 '31041',
 '41021',
 '41002',
 '41012',
 '41031',
 '41040',
 '51011',
 '51021',
 '51002',
 '51030',
 '51040',
 '01122',
 '01132',
 '01103',
 '01113',
 '01141',
 '11112',
 '11122',
 '11131',
 '11103',
 '11141',
 '21121',
 '21102',
 '21112',


In [64]:
winScenario = pd.DataFrame(winScenario, columns = ['Scenario'])
winScenario

,Scenario
0,00043
1,00024
2,00034
3,10033
4,10043
...,...
109,41120
110,41130
111,51110
112,51101


In [67]:
winScenario['VictoryPoint'] = winScenario['Scenario'].astype(str).str[0].astype(int)
winScenario['LongestRoad'] = winScenario['Scenario'].astype(str).str[1].astype(int)
winScenario['LargestArmy'] = winScenario['Scenario'].astype(str).str[2].astype(int)
winScenario['Settlement'] = winScenario['Scenario'].astype(str).str[3].astype(int)
winScenario['City'] = winScenario['Scenario'].astype(str).str[4].astype(int)
winScenario['TotalScore'] = (
    winScenario['VictoryPoint'] 
    + winScenario['LongestRoad']*2 
    + winScenario['LargestArmy']*2 
    + winScenario['Settlement']
    + winScenario['City']*2
)
winScenario

,Scenario,VictoryPoint,LongestRoad,LargestArmy,Settlement,City,TotalScore
0,00043,0,0,0,4,3,10
1,00024,0,0,0,2,4,10
2,00034,0,0,0,3,4,11
3,10033,1,0,0,3,3,10
4,10043,1,0,0,4,3,11
...,...,...,...,...,...,...,...
109,41120,4,1,1,2,0,10
110,41130,4,1,1,3,0,11
111,51110,5,1,1,1,0,10
112,51101,5,1,1,0,1,11


In [68]:
# Checking to see if anything is impossible, such as <= 1 settlement with no city
winScenario[(winScenario['Settlement'] <= 1) & (winScenario['City'] == 0)]

,Scenario,VictoryPoint,LongestRoad,LargestArmy,Settlement,City,TotalScore
111,51110,5,1,1,1,0,10


In [70]:
# Removing scenario 51101, this is not possible, as at game start we would always have 
winScenario = winScenario[winScenario['Scenario'] != '51101']
winScenario

,Scenario,VictoryPoint,LongestRoad,LargestArmy,Settlement,City,TotalScore
0,00043,0,0,0,4,3,10
1,00024,0,0,0,2,4,10
2,00034,0,0,0,3,4,11
3,10033,1,0,0,3,3,10
4,10043,1,0,0,4,3,11
...,...,...,...,...,...,...,...
108,41111,4,1,1,1,1,11
109,41120,4,1,1,2,0,10
110,41130,4,1,1,3,0,11
111,51110,5,1,1,1,0,10


In [84]:
# There is a special scenario, where the player has the 2nd longest road, and places the final 
# settlement to cut off the longest road holder and gain a final settlement and longest road, winning with 12 points
# This can happen when winning longestRoad = True and 1 settlement or more
winScenario[(winScenario['LongestRoad'] == 1) & (winScenario['Settlement'] >= 1) & (winScenario['TotalScore'] == 10)]

,Scenario,VictoryPoint,LongestRoad,LargestArmy,Settlement,City,TotalScore
57,01023,0,1,0,2,3,10
59,01042,0,1,0,4,2,10
62,11032,1,1,0,3,2,10
63,11013,1,1,0,1,3,10
67,21022,2,1,0,2,2,10
71,21041,2,1,0,4,1,10
72,31012,3,1,0,1,2,10
74,31031,3,1,0,3,1,10
77,41021,4,1,0,2,1,10
81,41040,4,1,0,4,0,10
